In [1]:
import numpy as np
from numpy.random import random as rnd 
import pandas as pd
from numpy.linalg import inv as ii

class parameters:
    def __init__(self,n):
        self.mu = rnd((n,1))
        self.F = rnd((n,n))
        self.R = rnd((1,1))
        self.q = rnd((n,n))
        self.Q = self.q.T@self.q
        self.beta00 = rnd((n,1))
        self.p00 = rnd((n,n))
        self.P00 = self.p00.T@self.p00

class data:
    def __init__(self,n):
        self.y = rnd((1,1))
        self.x = rnd((1,n))
        
class prediction:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.eta = rnd((1,1))
        self.f = rnd((1,1))
        
class updating:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.K = rnd((n,1))
        
class a_period:
    def __init__(self,t,model):
        self.t = t
        self.mymodel = model
        n = self.mymodel.n
        self.data = data(n)
        self.prd = prediction(n)
        self.upd = updating(n)
    def predict(self):
        par = self.mymodel.pars
        mu = par.mu
        F = par.F
        if self.t == 0: 
            b = par.beta00
            P = par.P00
        else:
            previous_period = self.mymodel.sample[self.t-1]
            b = previous_period.upd.beta
            P = previous_period.upd.P
        self.prd.beta = mu + F @ b
        self.prd.P = (F @ P) @ F.T + par.Q
        x = self.data.x
        self.prd.eta = self.data.y - x @ self.prd.beta
        self.prd.f = (x @ self.prd.P) @ x.T + par.R
    def update(self):
        P = self.prd.P
        x = self.data.x
        K = (P @ x.T) @ ii(self.prd.f)
        self.upd.K = K
        self.upd.beta = self.prd.beta + K @ self.prd.eta
        self.upd.P = P - (K @ x) @ P
            
class the_model:
    def __init__(self,datafile):
        self.datafile = datafile
        df = pd.read_csv(datafile)
        T,n = df.shape
        n = n - 1
        self.n = n
        self.T = T
        self.pars = parameters(n)
        self.sample = []
        for t in range(T): 
            new_period = a_period(t,self)
            new_period.data.y = np.array([[ df.iloc[t,0] ]])
            new_period.data.x = df.iloc[t,1:].values.reshape((1,n))
            self.sample.append(new_period)
    def run(self):
        for t in range(self.T):
            self.sample[t].predict()
            self.sample[t].update()
        
        

In [2]:
ibm = the_model("IBM.csv")

In [3]:
ibm.run()

In [4]:
for aperiod in ibm.sample: print(aperiod.upd.beta.T)

[[0.75746452 1.00740751 0.86714058]]
[[1.42770917 2.26842493 1.52754914]]
[[2.29930609 3.29558536 1.91494411]]
[[3.98198873 5.22369849 2.79657962]]
[[6.19159403 7.78365993 3.94873809]]
[[4.71639188 6.05031192 3.16035964]]
[[6.81441487 8.4977946  4.27749287]]
[[4.94836932 6.32163734 3.28102226]]
[[5.13632677 6.55607217 3.39766615]]
[[6.91546954 8.61604388 4.32855947]]
[[5.59382995 7.07893589 3.6324501 ]]
[[ 8.70458367 10.67666063  5.2566303 ]]
[[0.42904563 1.11471092 0.93967694]]
[[0.79358219 1.50956023 1.10476386]]
[[1.91686011 2.83881613 1.72807484]]
[[3.50757503 4.68793112 2.55691832]]
[[4.45008095 5.75950684 3.03454788]]
[[5.19268482 6.62209155 3.42823341]]
[[7.6288905  9.4317998  4.69378551]]
[[11.17952558 13.5314666   6.54129053]]
[[12.07110429 14.55948786  7.0045735 ]]
[[ 8.28362302 10.18909374  5.03430909]]
[[4.27942517 5.56193231 2.9454807 ]]
[[0.39248299 1.08392887 0.92881209]]
[[0.71302989 1.4428378  1.0879022 ]]
[[1.80125474 2.71350267 1.66820528]]
[[3.28104989 4.42296781 2.

$\mathcal{L} = -\frac{1}{2}\sum_{t-1}^T \Big(\ln(2\pi f_{t|t-1}) + \eta'_{t|t-1}f^{-1}_{t|t-1}\eta_{t|t-1}\Big).$

In [ ]:
#self.ll = math.log(2 * math.pi * f) + (eta.T @ inv(f)) @ eta

In [9]:
import numpy as np
from numpy.random import random as rnd 
import pandas as pd
from numpy.linalg import inv
import math

class parameters:
    def __init__(self,n):
        self.mu = rnd((n,1))
        self.F = rnd((n,n))
        self.R = rnd((1,1))
        self.q = rnd((n,n))
        self.Q = self.q.T@self.q
        self.beta00 = rnd((n,1))
        self.p00 = rnd((n,n))
        self.P00 = self.p00.T@self.p00

class data:
    def __init__(self,n):
        self.y = rnd((1,1))
        self.x = rnd((1,n))
        
class prediction:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.eta = rnd((1,1))
        self.f = rnd((1,1))
        
class updating:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.K = rnd((n,1))
        
class a_period:
    def __init__(self,t,model):
        self.t = t
        self.mymodel = model
        n = self.mymodel.n
        self.data = data(n)
        self.prd = prediction(n)
        self.upd = updating(n)
    def predict(self):
        par = self.mymodel.pars
        mu = par.mu
        F = par.F
        if self.t == 0: 
            b = par.beta00
            P = par.P00
        else:
            previous_period = self.mymodel.sample[self.t-1]
            b = previous_period.upd.beta
            P = previous_period.upd.P
        self.prd.beta = mu + F @ b
        self.prd.P = (F @ P) @ F.T + par.Q
        x = self.data.x
        eta = self.data.y - x @ self.prd.beta
        f = (x @ self.prd.P) @ x.T + par.R
        self.prd.eta = eta
        self.prd.f = f
        self.ll = float( math.log(2 * math.pi * f) + (eta.T @ inv(f)) @ eta )
        
    def update(self):
        P = self.prd.P
        x = self.data.x
        K = (P @ x.T) @ ii(self.prd.f)
        self.upd.K = K
        self.upd.beta = self.prd.beta + K @ self.prd.eta
        self.upd.P = P - (K @ x) @ P
            
class the_model:
    def __init__(self,datafile):
        self.datafile = datafile
        df = pd.read_csv(datafile)
        T,n = df.shape
        n = n - 1
        self.n = n
        self.T = T
        self.pars = parameters(n)
        self.sample = []
        for t in range(T): 
            new_period = a_period(t,self)
            new_period.data.y = np.array([[ df.iloc[t,0] ]])
            new_period.data.x = df.iloc[t,1:].values.reshape((1,n))
            self.sample.append(new_period)
    def run(self):
        self.ll = 0
        for t in range(self.T):
            period_t = self.sample[t]
            period_t.predict()
            period_t.update()
            self.ll = self.ll + period_t.ll
        self.ll = -0.5 * self.ll
        
        

In [10]:
ibm = the_model("IBM.csv")

In [11]:
ibm.run()

In [12]:
ibm.ll

-548.0804221525112

In [13]:
for aperiod in ibm.sample: print(aperiod.upd.beta.T)

[[1.35379248 0.44646624 1.38415563]]
[[2.7486898  2.29068601 2.7718455 ]]
[[4.64741265 4.1296778  4.15988187]]
[[8.60845482 7.67408013 7.29951405]]
[[11.14827401 10.04745313  9.39200879]]
[[5.87095283 5.16528888 5.1523433 ]]
[[8.98788875 8.06370567 7.64377752]]
[[3.12892034 2.58778005 2.92465159]]
[[2.95734805 2.43165966 2.79022562]]
[[6.20249045 5.45800614 5.40231018]]
[[4.58293394 3.94638572 4.09794958]]
[[8.40270838 7.50871067 7.17491664]]
[[0.68899782 0.31654524 0.96150345]]
[[1.06397861 0.66455852 1.26429112]]
[[2.58893291 2.08877756 2.49207083]]
[[5.10904744 4.43746971 4.52129667]]
[[4.2293337  3.61701585 3.8132594 ]]
[[4.36595196 3.74501075 3.92359018]]
[[7.58955859 6.75001466 6.519586  ]]
[[7.18960854 6.37769399 6.19763193]]
[[4.88082747 4.22472949 4.33778751]]
[[2.99515685 2.46638373 2.81886657]]
[[0.96259105 0.57116388 1.18156498]]
[[ 0.29130928 -0.05456052  0.6408171 ]]
[[0.66944422 0.29731537 0.94522677]]
[[1.82821107 1.37839447 1.87892568]]
[[3.7058824  3.12899557 3.391199

<font face = "Georgia" size="+1"> We obtained the values of the time-varying betas on the three Fama and French factors that are not very meaningful because we used random parameters</font> $F, \mu, R, Q, \beta_{0|0}$, <font face = "Georgia" size="+1">and</font> $P_{0|0}$.

<font face = "Georgia" size="+1">To obtain more meaningful betas, we need to <strong>estimate</strong> those parameters by <strong>maximizing</strong> the log-likelihood function, which we call <tt>self.ll</tt>.</font>
    
<font face = "Georgia" size="+1">To maximize this funcion, we are going to use the <tt>scipy</tt>'s <tt> minimize</tt> function to minimize the <strong>negative</strong> value of the <tt>self.ll</tt>.</font>

In [ ]:
from scipy.optimize import minimize

minimize(
    fun = # The function that we would like to minimize;
    x0 = # Starting values of the parameters;
    method = # The numerical method to be used for minimization;
    constraints = # Define the valid sets of the values of the parameters;
    tol = # Tolerance for convergense;
    options = # Options for the minimization process; e.g., the maximum number of 
              # iterations to be attempted.
)

In [93]:
import numpy as np
from numpy.random import random as rnd 
import pandas as pd
from numpy.linalg import inv
import math

class parameters:
    def __init__(self,n):
        self.mu = rnd((n,1))
        self.F = rnd((n,n))
        self.R = rnd((1,1))
        self.q = rnd((n,n))
        self.Q = self.q.T@self.q
        self.beta00 = rnd((n,1))
        self.p00 = rnd((n,n))
        self.P00 = self.p00.T@self.p00
        self.parnames = list(self.__dict__.keys())
        self.parnames.sort()
        self.do_not_pack = ['Q','P00']
    def pack (self):
        big_column = np.array([[]]).T
        for aname in self.parnames: 
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            matrix = matrix.reshape((-1,1),order = "F") 
            big_column = np.concatenate((big_column,matrix),axis = 0)
        return big_column
            
class data:
    def __init__(self,n):
        self.y = rnd((1,1))
        self.x = rnd((1,n))
        
class prediction:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.eta = rnd((1,1))
        self.f = rnd((1,1))
        
class updating:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.K = rnd((n,1))
        
class a_period:
    def __init__(self,t,model):
        self.t = t
        self.mymodel = model
        n = self.mymodel.n
        self.data = data(n)
        self.prd = prediction(n)
        self.upd = updating(n)
    def predict(self):
        par = self.mymodel.pars
        mu = par.mu
        F = par.F
        if self.t == 0: 
            b = par.beta00
            P = par.P00
        else:
            previous_period = self.mymodel.sample[self.t-1]
            b = previous_period.upd.beta
            P = previous_period.upd.P
        self.prd.beta = mu + F @ b
        self.prd.P = (F @ P) @ F.T + par.Q
        x = self.data.x
        eta = self.data.y - x @ self.prd.beta
        f = (x @ self.prd.P) @ x.T + par.R
        self.prd.eta = eta
        self.prd.f = f
        self.ll = float( math.log(2 * math.pi * f) + (eta.T @ inv(f)) @ eta )
        
    def update(self):
        P = self.prd.P
        x = self.data.x
        K = (P @ x.T) @ ii(self.prd.f)
        self.upd.K = K
        self.upd.beta = self.prd.beta + K @ self.prd.eta
        self.upd.P = P - (K @ x) @ P
            
class the_model:
    def __init__(self,datafile):
        self.datafile = datafile
        df = pd.read_csv(datafile)
        T,n = df.shape
        n = n - 1
        self.n = n
        self.T = T
        self.pars = parameters(n)
        self.sample = []
        for t in range(T): 
            new_period = a_period(t,self)
            new_period.data.y = np.array([[ df.iloc[t,0] ]])
            new_period.data.x = df.iloc[t,1:].values.reshape((1,n))
            self.sample.append(new_period)
    def run(self):
        self.ll = 0
        for t in range(self.T):
            period_t = self.sample[t]
            period_t.predict()
            period_t.update()
            self.ll = self.ll + period_t.ll
        self.ll = -0.5 * self.ll
        
        

In [94]:
ibm = the_model('IBM.csv')

In [95]:
ibm.pars.pack()

array([[0.30694643],
       [0.93702452],
       [0.53656699],
       [0.02323843],
       [0.27746739],
       [0.49101502],
       [0.86633124],
       [0.98659775],
       [0.93083993],
       [0.49647692],
       [0.32641551],
       [0.48217113],
       [0.64233266],
       [0.59422904],
       [0.83884237],
       [0.81291675],
       [0.34668714],
       [0.59471478],
       [0.04274704],
       [0.26985976],
       [0.86019759],
       [0.11229545],
       [0.41224117],
       [0.32167221],
       [0.25471399],
       [0.41412691],
       [0.17074473],
       [0.22380124],
       [0.55123425],
       [0.4307294 ],
       [0.97811023],
       [0.90922264],
       [0.17495924],
       [0.40007631]])

In [96]:
ibm.pars.Q

array([[0.25074185, 0.52072799, 0.4959445 ],
       [0.52072799, 1.44608665, 0.96787349],
       [0.4959445 , 0.96787349, 1.0173576 ]])

In [132]:
import numpy as np
from numpy.random import random as rnd 
import pandas as pd
from numpy.linalg import inv
import math

class parameters:
    def __init__(self,n):
        self.mu = rnd((n,1))
        self.F = rnd((n,n))
        self.R = rnd((1,1))
        self.q = rnd((n,n))
        self.Q = self.q.T@self.q
        self.beta00 = rnd((n,1))
        self.p00 = rnd((n,n))
        self.P00 = self.p00.T@self.p00
        self.parnames = list(self.__dict__.keys())
        self.parnames.sort()
        self.do_not_pack = ['Q','P00']
    def pack(self):
        big_column = np.array([[]]).T
        for aname in self.parnames: 
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            matrix = matrix.reshape((-1,1),order = "F") 
            big_column = np.concatenate((big_column,matrix),axis = 0)
        return big_column
    def unpack(self, big_column):
        position = 0
        for aname in self.parnames:
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            nrow, ncol = matrix.shape
            new_position = position + nrow*ncol
            new_matrix = big_column[position : new_position]
            new_matrix = new_matrix.reshape((nrow, ncol), order = 'F')
            setattr(self, aname, new_matrix)
            position = new_position
        self.Q = self.q.T@self.q
        self.P00 = self.p00.T@self.p00
            
class data:
    def __init__(self,n):
        self.y = rnd((1,1))
        self.x = rnd((1,n))
        
class prediction:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.eta = rnd((1,1))
        self.f = rnd((1,1))
        
class updating:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.K = rnd((n,1))
        
class a_period:
    def __init__(self,t,model):
        self.t = t
        self.mymodel = model
        n = self.mymodel.n
        self.data = data(n)
        self.prd = prediction(n)
        self.upd = updating(n)
    def predict(self):
        par = self.mymodel.pars
        mu = par.mu
        F = par.F
        if self.t == 0: 
            b = par.beta00
            P = par.P00
        else:
            previous_period = self.mymodel.sample[self.t-1]
            b = previous_period.upd.beta
            P = previous_period.upd.P
        self.prd.beta = mu + F @ b
        self.prd.P = (F @ P) @ F.T + par.Q
        x = self.data.x
        eta = self.data.y - x @ self.prd.beta
        f = (x @ self.prd.P) @ x.T + par.R
        self.prd.eta = eta
        self.prd.f = f
        self.ll = float( math.log(2 * math.pi * f) + (eta.T @ inv(f)) @ eta )
        
    def update(self):
        P = self.prd.P
        x = self.data.x
        K = (P @ x.T) @ ii(self.prd.f)
        self.upd.K = K
        self.upd.beta = self.prd.beta + K @ self.prd.eta
        self.upd.P = P - (K @ x) @ P
            
class the_model:
    def __init__(self,datafile):
        self.datafile = datafile
        df = pd.read_csv(datafile)
        T,n = df.shape
        n = n - 1
        self.n = n
        self.T = T
        self.pars = parameters(n)
        self.sample = []
        for t in range(T): 
            new_period = a_period(t,self)
            new_period.data.y = np.array([[ df.iloc[t,0] ]])
            new_period.data.x = df.iloc[t,1:].values.reshape((1,n))
            self.sample.append(new_period)
    def run(self):
        self.ll = 0
        for t in range(self.T):
            period_t = self.sample[t]
            period_t.predict()
            period_t.update()
            self.ll = self.ll + period_t.ll
        self.ll = -0.5 * self.ll
        
        

In [133]:
ibm = the_model("IBM.csv")

In [134]:
verylong = ibm.pars.pack()

In [135]:
verylong

array([[0.32767097],
       [0.79334154],
       [0.29986159],
       [0.27160096],
       [0.75801044],
       [0.31307124],
       [0.97263743],
       [0.57881373],
       [0.39754809],
       [0.25472049],
       [0.62884929],
       [0.34376763],
       [0.96838304],
       [0.79968369],
       [0.035909  ],
       [0.14406206],
       [0.81950696],
       [0.64691175],
       [0.23386313],
       [0.27244569],
       [0.56368905],
       [0.28537286],
       [0.13650523],
       [0.85890557],
       [0.25998264],
       [0.11067723],
       [0.18537215],
       [0.78338975],
       [0.67567166],
       [0.9141737 ],
       [0.79826711],
       [0.25963651],
       [0.14361388],
       [0.81798706]])

In [137]:
from copy import deepcopy as dc

In [138]:
original_parameters = dc(ibm.pars)

In [139]:
original_parameters.__dict__

{'mu': array([[0.79968369],
        [0.035909  ],
        [0.14406206]]),
 'F': array([[0.32767097, 0.27160096, 0.97263743],
        [0.79334154, 0.75801044, 0.57881373],
        [0.29986159, 0.31307124, 0.39754809]]),
 'R': array([[0.25472049]]),
 'q': array([[0.11067723, 0.67567166, 0.25963651],
        [0.18537215, 0.9141737 , 0.14361388],
        [0.78338975, 0.79826711, 0.81798706]]),
 'Q': array([[0.66031178, 0.86959808, 0.69616054],
        [0.86959808, 1.92947613, 0.95968923],
        [0.69616054, 0.95968923, 0.75713889]]),
 'beta00': array([[0.62884929],
        [0.34376763],
        [0.96838304]]),
 'p00': array([[0.81950696, 0.27244569, 0.13650523],
        [0.64691175, 0.56368905, 0.85890557],
        [0.23386313, 0.28537286, 0.25998264]]),
 'P00': array([[1.14477844, 0.65466641, 0.72830345],
        [0.65466641, 0.47340968, 0.59553792],
        [0.72830345, 0.59553792, 0.82394343]]),
 'parnames': ['F', 'P00', 'Q', 'R', 'beta00', 'mu', 'p00', 'q'],
 'do_not_pack': ['Q', 'P0

In [140]:
new_parameters = parameters(3)

In [141]:
original_parameters.F

array([[0.32767097, 0.27160096, 0.97263743],
       [0.79334154, 0.75801044, 0.57881373],
       [0.29986159, 0.31307124, 0.39754809]])

In [142]:
new_parameters.F

array([[0.53681764, 0.39849255, 0.40210815],
       [0.41594359, 0.63826622, 0.34647715],
       [0.55282848, 0.03367481, 0.52700789]])

In [143]:
original_parameters.Q

array([[0.66031178, 0.86959808, 0.69616054],
       [0.86959808, 1.92947613, 0.95968923],
       [0.69616054, 0.95968923, 0.75713889]])

In [144]:
new_parameters.Q

array([[0.67152873, 0.3852717 , 0.46105515],
       [0.3852717 , 0.90460712, 0.76896432],
       [0.46105515, 0.76896432, 0.73049711]])

In [145]:
new_parameters.unpack(verylong)

In [146]:
original_parameters.F

array([[0.32767097, 0.27160096, 0.97263743],
       [0.79334154, 0.75801044, 0.57881373],
       [0.29986159, 0.31307124, 0.39754809]])

In [147]:
new_parameters.F

array([[0.32767097, 0.27160096, 0.97263743],
       [0.79334154, 0.75801044, 0.57881373],
       [0.29986159, 0.31307124, 0.39754809]])

In [148]:
original_parameters.Q

array([[0.66031178, 0.86959808, 0.69616054],
       [0.86959808, 1.92947613, 0.95968923],
       [0.69616054, 0.95968923, 0.75713889]])

In [149]:
new_parameters.Q

array([[0.66031178, 0.86959808, 0.69616054],
       [0.86959808, 1.92947613, 0.95968923],
       [0.69616054, 0.95968923, 0.75713889]])

In [150]:
original_parameters.Q == new_parameters.Q

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [152]:
for aname in original_parameters.parnames:
    print ('+++++ '+aname+' +++++')
    print (getattr(original_parameters, aname) == getattr(new_parameters, aname))

+++++ F +++++
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
+++++ P00 +++++
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
+++++ Q +++++
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
+++++ R +++++
[[ True]]
+++++ beta00 +++++
[[ True]
 [ True]
 [ True]]
+++++ mu +++++
[[ True]
 [ True]
 [ True]]
+++++ p00 +++++
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
+++++ q +++++
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


In [153]:
original_parameters = dc(ibm.pars)
new_parameters = parameters(3)

In [154]:
for aname in original_parameters.parnames:
    print ('+++++ '+aname+' +++++')
    print (getattr(original_parameters, aname) == getattr(new_parameters, aname))

+++++ F +++++
[[False False False]
 [False False False]
 [False False False]]
+++++ P00 +++++
[[False False False]
 [False False False]
 [False False False]]
+++++ Q +++++
[[False False False]
 [False False False]
 [False False False]]
+++++ R +++++
[[False]]
+++++ beta00 +++++
[[False]
 [False]
 [False]]
+++++ mu +++++
[[False]
 [False]
 [False]]
+++++ p00 +++++
[[False False False]
 [False False False]
 [False False False]]
+++++ q +++++
[[False False False]
 [False False False]
 [False False False]]


In [155]:
new_parameters.unpack(verylong)

In [156]:
for aname in original_parameters.parnames:
    print ('+++++ '+aname+' +++++')
    print (getattr(original_parameters, aname) == getattr(new_parameters, aname))

+++++ F +++++
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
+++++ P00 +++++
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
+++++ Q +++++
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
+++++ R +++++
[[ True]]
+++++ beta00 +++++
[[ True]
 [ True]
 [ True]]
+++++ mu +++++
[[ True]
 [ True]
 [ True]]
+++++ p00 +++++
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
+++++ q +++++
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


We should NOT forget that the variance of errors, R, can not be negative! So, we need to make sure it is not negative.

In [ ]:
import numpy as np
from numpy.random import random as rnd 
import pandas as pd
from numpy.linalg import inv
import math

class parameters:
    def __init__(self,n):
        self.mu = rnd((n,1))
        self.F = rnd((n,n))
        self.r = rnd((1,1))
        self.R = self.r.T@self.r
        self.q = rnd((n,n))
        self.Q = self.q.T@self.q
        self.beta00 = rnd((n,1))
        self.p00 = rnd((n,n))
        self.P00 = self.p00.T@self.p00
        self.parnames = list(self.__dict__.keys())
        self.parnames.sort()
        self.do_not_pack = ['R','Q','P00']
    def pack(self):
        big_column = np.array([[]]).T
        for aname in self.parnames: 
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            matrix = matrix.reshape((-1,1),order = "F") 
            big_column = np.concatenate((big_column,matrix),axis = 0)
        return big_column
    def unpack(self, big_column):
        position = 0
        for aname in self.parnames:
            if aname in self.do_not_pack: continue
            matrix = getattr(ibm.pars,aname) 
            nrow, ncol = matrix.shape
            new_position = position + nrow*ncol
            new_matrix = big_column[position : new_position]
            new_matrix = new_matrix.reshape((nrow, ncol), order = 'F')
            setattr(self, aname, new_matrix)
            position = new_position
        self.R = self.r.T@self.r
        self.Q = self.q.T@self.q
        self.P00 = self.p00.T@self.p00
            
class data:
    def __init__(self,n):
        self.y = rnd((1,1))
        self.x = rnd((1,n))
        
class prediction:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.eta = rnd((1,1))
        self.f = rnd((1,1))
        
class updating:
    def __init__(self,n):
        self.beta = rnd((n,1))
        self.P = rnd((n,n))
        self.K = rnd((n,1))
        
class a_period:
    def __init__(self,t,model):
        self.t = t
        self.mymodel = model
        n = self.mymodel.n
        self.data = data(n)
        self.prd = prediction(n)
        self.upd = updating(n)
    def predict(self):
        par = self.mymodel.pars
        mu = par.mu
        F = par.F
        if self.t == 0: 
            b = par.beta00
            P = par.P00
        else:
            previous_period = self.mymodel.sample[self.t-1]
            b = previous_period.upd.beta
            P = previous_period.upd.P
        self.prd.beta = mu + F @ b
        self.prd.P = (F @ P) @ F.T + par.Q
        x = self.data.x
        eta = self.data.y - x @ self.prd.beta
        f = (x @ self.prd.P) @ x.T + par.R
        self.prd.eta = eta
        self.prd.f = f
        self.ll = float( math.log(2 * math.pi * f) + (eta.T @ inv(f)) @ eta )
        
    def update(self):
        P = self.prd.P
        x = self.data.x
        K = (P @ x.T) @ ii(self.prd.f)
        self.upd.K = K
        self.upd.beta = self.prd.beta + K @ self.prd.eta
        self.upd.P = P - (K @ x) @ P
            
class the_model:
    def __init__(self,datafile):
        self.datafile = datafile
        df = pd.read_csv(datafile)
        T,n = df.shape
        n = n - 1
        self.n = n
        self.T = T
        self.pars = parameters(n)
        self.sample = []
        for t in range(T): 
            new_period = a_period(t,self)
            new_period.data.y = np.array([[ df.iloc[t,0] ]])
            new_period.data.x = df.iloc[t,1:].values.reshape((1,n))
            self.sample.append(new_period)
    def run(self):
        self.ll = 0
        for t in range(self.T):
            period_t = self.sample[t]
            period_t.predict()
            period_t.update()
            self.ll = self.ll + period_t.ll
        self.ll = -0.5 * self.ll
        
        

<h3>Scratchwork:</h3>

In [15]:
ibm.pars.mu

array([[0.87152552],
       [0.08092436],
       [0.9425948 ]])

In [16]:
ibm.pars.F

array([[0.96439256, 0.73159145, 0.16776893],
       [0.55231926, 0.39272163, 0.92201489],
       [0.9891472 , 0.24715212, 0.26765849]])

In [17]:
F = ibm.pars.F

In [18]:
F

array([[0.96439256, 0.73159145, 0.16776893],
       [0.55231926, 0.39272163, 0.92201489],
       [0.9891472 , 0.24715212, 0.26765849]])

In [19]:
F.shape

(3, 3)

In [20]:
F.reshape((9,1))

array([[0.96439256],
       [0.73159145],
       [0.16776893],
       [0.55231926],
       [0.39272163],
       [0.92201489],
       [0.9891472 ],
       [0.24715212],
       [0.26765849]])

In [21]:
F.reshape((1,9))

array([[0.96439256, 0.73159145, 0.16776893, 0.55231926, 0.39272163,
        0.92201489, 0.9891472 , 0.24715212, 0.26765849]])

In [22]:
F.reshape((1,-1))

array([[0.96439256, 0.73159145, 0.16776893, 0.55231926, 0.39272163,
        0.92201489, 0.9891472 , 0.24715212, 0.26765849]])

In [23]:
F.reshape((-1,9))

array([[0.96439256, 0.73159145, 0.16776893, 0.55231926, 0.39272163,
        0.92201489, 0.9891472 , 0.24715212, 0.26765849]])

In [24]:
F.reshape((-1,1))

array([[0.96439256],
       [0.73159145],
       [0.16776893],
       [0.55231926],
       [0.39272163],
       [0.92201489],
       [0.9891472 ],
       [0.24715212],
       [0.26765849]])

In [26]:
F

array([[0.96439256, 0.73159145, 0.16776893],
       [0.55231926, 0.39272163, 0.92201489],
       [0.9891472 , 0.24715212, 0.26765849]])

In [27]:
Q = ibm.pars.Q

In [28]:
Q

array([[0.35974823, 0.26570366, 0.15585106],
       [0.26570366, 0.3623482 , 0.12209725],
       [0.15585106, 0.12209725, 0.07960544]])

In [29]:
Q.reshape((-1,1))

array([[0.35974823],
       [0.26570366],
       [0.15585106],
       [0.26570366],
       [0.3623482 ],
       [0.12209725],
       [0.15585106],
       [0.12209725],
       [0.07960544]])

In [30]:
Q

array([[0.35974823, 0.26570366, 0.15585106],
       [0.26570366, 0.3623482 , 0.12209725],
       [0.15585106, 0.12209725, 0.07960544]])

In [31]:
Q.reshape((-1,1), order='F') # that is, the Fortran order.

array([[0.35974823],
       [0.26570366],
       [0.15585106],
       [0.26570366],
       [0.3623482 ],
       [0.12209725],
       [0.15585106],
       [0.12209725],
       [0.07960544]])

In [33]:
mu = ibm.pars.mu

In [34]:
mu

array([[0.87152552],
       [0.08092436],
       [0.9425948 ]])

In [35]:
cQ = Q.reshape((-1,1), order='F')

In [36]:
cQ

array([[0.35974823],
       [0.26570366],
       [0.15585106],
       [0.26570366],
       [0.3623482 ],
       [0.12209725],
       [0.15585106],
       [0.12209725],
       [0.07960544]])

In [37]:
np.concatenate((mu,cQ),axis = 0)

array([[0.87152552],
       [0.08092436],
       [0.9425948 ],
       [0.35974823],
       [0.26570366],
       [0.15585106],
       [0.26570366],
       [0.3623482 ],
       [0.12209725],
       [0.15585106],
       [0.12209725],
       [0.07960544]])

In [39]:
ibm.pars.__dict__

{'mu': array([[0.87152552],
        [0.08092436],
        [0.9425948 ]]),
 'F': array([[0.96439256, 0.73159145, 0.16776893],
        [0.55231926, 0.39272163, 0.92201489],
        [0.9891472 , 0.24715212, 0.26765849]]),
 'R': array([[0.74705871]]),
 'q': array([[0.28648225, 0.53967305, 0.09969657],
        [0.27276258, 0.2488123 , 0.21605008],
        [0.4508622 , 0.09588347, 0.15161924]]),
 'Q': array([[0.35974823, 0.26570366, 0.15585106],
        [0.26570366, 0.3623482 , 0.12209725],
        [0.15585106, 0.12209725, 0.07960544]]),
 'beta00': array([[0.54867541],
        [0.21814241],
        [0.30831502]]),
 'p00': array([[0.83396823, 0.20888618, 0.9943604 ],
        [0.35327771, 0.21622132, 0.58249905],
        [0.43421853, 0.19457631, 0.1212151 ]]),
 'P00': array([[1.00885389, 0.33507925, 1.08768276],
        [0.33507925, 0.12824504, 0.35724245],
        [1.08768276, 0.35724245, 1.34275086]])}

In [40]:
ibm.__dict__

{'datafile': 'IBM.csv',
 'n': 3,
 'T': 405,
 'pars': <__main__.parameters at 0x7f3edf9b5e90>,
 'sample': [<__main__.a_period at 0x7f3edf9b5d10>,
 'll': -548.0804221525112}

In [41]:
ibm.pars.__dict__.keys()

dict_keys(['mu', 'F', 'R', 'q', 'Q', 'beta00', 'p00', 'P00'])

In [42]:
list(ibm.pars.__dict__.keys())

['mu', 'F', 'R', 'q', 'Q', 'beta00', 'p00', 'P00']

In [43]:
parnames = list(ibm.pars.__dict__.keys())

In [44]:
parnames

['mu', 'F', 'R', 'q', 'Q', 'beta00', 'p00', 'P00']

In [45]:
parnames.sort()

In [46]:
parnames

['F', 'P00', 'Q', 'R', 'beta00', 'mu', 'p00', 'q']

In [47]:
parnames[0]

'F'

In [49]:
parnames[1]

'P00'

In [50]:
parameter_0 = parnames[0]

In [51]:
parameter_0

'F'

In [52]:
ibm.pars.F

array([[0.96439256, 0.73159145, 0.16776893],
       [0.55231926, 0.39272163, 0.92201489],
       [0.9891472 , 0.24715212, 0.26765849]])

In [53]:
ibm.pars.parameter_0

AttributeError: 'parameters' object has no attribute 'parameter_0'

In [54]:
# The "Get Attribute function":
getattr(ibm.pars, 'F')

array([[0.96439256, 0.73159145, 0.16776893],
       [0.55231926, 0.39272163, 0.92201489],
       [0.9891472 , 0.24715212, 0.26765849]])

In [55]:
ibm.pars.F

array([[0.96439256, 0.73159145, 0.16776893],
       [0.55231926, 0.39272163, 0.92201489],
       [0.9891472 , 0.24715212, 0.26765849]])

In [56]:
getattr(ibm.pars, parameter_0)

array([[0.96439256, 0.73159145, 0.16776893],
       [0.55231926, 0.39272163, 0.92201489],
       [0.9891472 , 0.24715212, 0.26765849]])

That is, this is saying: "<em>Get attribute from <tt><font color="blue">ibm.pars</font></tt>, whose name is specified in the variable <tt><font color="blue">parameter_0</font></tt>.</em>"

We can also set an attribute vlue by using a similar function.

In [60]:
newF = 'This is a new value of F'

In [61]:
newF

'This is a new value of F'

In [62]:
ibm.pars.F

array([[0.96439256, 0.73159145, 0.16776893],
       [0.55231926, 0.39272163, 0.92201489],
       [0.9891472 , 0.24715212, 0.26765849]])

In [63]:
setattr(ibm.pars, parameter_0, newF)

In the object by the name <font color="blue"><tt>ibm.pars</tt></font>, we set the attribute by the name <font color="blue"><tt>parameter_0</tt></font> to be equal to the new value <font color="blue"><tt>newF</tt></font>.

In [64]:
ibm.pars.F

'This is a new value of F'

In [65]:
getattr(ibm.pars, parameter_0)

'This is a new value of F'

In [66]:
parnames

['F', 'P00', 'Q', 'R', 'beta00', 'mu', 'p00', 'q']

In [67]:
big_column = np.array([[]])

In [68]:
big_column

array([], shape=(1, 0), dtype=float64)

In [70]:
big_column.T

array([], shape=(0, 1), dtype=float64)

In [71]:
big_column = np.array([[]]).T

In [72]:
big_column

array([], shape=(0, 1), dtype=float64)

In [73]:
np.concatenate((big_column,mu),axis = 0)

array([[0.87152552],
       [0.08092436],
       [0.9425948 ]])

In [74]:
mu

array([[0.87152552],
       [0.08092436],
       [0.9425948 ]])

In [75]:
parnames

['F', 'P00', 'Q', 'R', 'beta00', 'mu', 'p00', 'q']

In [76]:
for aname in parnames:
    print (aname)

F
P00
Q
R
beta00
mu
p00
q


In [77]:
big_column = np.array([[]]).T # Set up an empty column.
for aname in parnames: # Loop over all parameter names.
    matrix = getattr(ibm.pars,aname) # Extract the value of the matrix by the name aname
    matrix = matrix.reshape((-1,1),order = "F") # Turn that matrix into a column.
    big_column = np.concatenate((big_column,matrix),axis = 0) # Concat it to the big_column.

AttributeError: 'str' object has no attribute 'reshape'

In [81]:
aname = 'F'

In [82]:
matrix = getattr(ibm.pars,aname)

In [83]:
matrix

'This is a new value of F'

In [84]:
matrix.reshape((-1,1),order = "F")

AttributeError: 'str' object has no attribute 'reshape'

In [86]:
n = 3
ibm.pars.F = rnd((n,n))

In [87]:
big_column = np.array([[]]).T # Set up an empty column.
for aname in parnames: # Loop over all parameter names.
    matrix = getattr(ibm.pars,aname) # Extract the value of the matrix by the name aname
    matrix = matrix.reshape((-1,1),order = "F") # Turn that matrix into a column.
    big_column = np.concatenate((big_column,matrix),axis = 0) # Concat it to the big_column.

In [88]:
big_column

array([[0.52269651],
       [0.27455729],
       [0.50837047],
       [0.90670022],
       [0.88888001],
       [0.37969602],
       [0.87493473],
       [0.55270639],
       [0.31258573],
       [1.00885389],
       [0.33507925],
       [1.08768276],
       [0.33507925],
       [0.12824504],
       [0.35724245],
       [1.08768276],
       [0.35724245],
       [1.34275086],
       [0.35974823],
       [0.26570366],
       [0.15585106],
       [0.26570366],
       [0.3623482 ],
       [0.12209725],
       [0.15585106],
       [0.12209725],
       [0.07960544],
       [0.74705871],
       [0.54867541],
       [0.21814241],
       [0.30831502],
       [0.87152552],
       [0.08092436],
       [0.9425948 ],
       [0.83396823],
       [0.35327771],
       [0.43421853],
       [0.20888618],
       [0.21622132],
       [0.19457631],
       [0.9943604 ],
       [0.58249905],
       [0.1212151 ],
       [0.28648225],
       [0.27276258],
       [0.4508622 ],
       [0.53967305],
       [0.248